In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name_or_path = ''
lora_path = ''

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)

In [ ]:

print(f"Loading LoRA weights from {lora_path}")
model = PeftModel.from_pretrained(model, lora_path)
print(f"Merging weights")
model = model.merge_and_unload()
print("Done")

In [ ]:
def generate_response(
    prompt, 
    do_sample:bool = True, 
    top_p:float=0.95, 
    temperature:float=0.7, 
    num_beams:int = 2,
    max_length:int = 512):

    with torch.inference_mode():
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        output = model.generate(
            input_ids, 
            num_return_sequences=1, 
            num_beams=num_beams,
            temperature=temperature, 
            do_sample=do_sample, 
            top_p=top_p,
            max_length=max_length)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [ ]:
prompt = '''
Dưới đây là hướng dẫn mô tả bài toán. \
Viết câu trả lời hoàn thành yêu cầu một cách thích hợp\n\
### Câu hỏi: {question}\n{choices}\n\
### Trả lời: 
'''

question = 'Giá trị của biểu thức 47,2 ${\\times}$ 5 + 107,1 : 2,5 là:'
choices = [
            "A. 137,24",
            "B. 1372,4",
            "C. 278,84",
            "D. 27,884"
         ]
choices = '\n'.join(choices)
prompt = prompt.format(question=question, choices = choices)

In [ ]:
generate_args = {
    "num_beams": 2,
    "temperature": 0.7, 
    "do_sample": True, 
    "top_p": 0.95,
    "max_length": 512,
}

response = generate_response(prompt, **generate_args)
parsed_response = response[len(prompt):]

print(parsed_response)